# $\mathbb{Z}_2$ Gauge Theory on a Square Lattice

In this notebook we build a simple **$\mathbb{Z}_2$ lattice gauge theory** on a 2D square lattice.  
We will:

1. Define the **link variables** on a lattice.
2. Construct the **plaquette values**.
3. Compute the **Hamiltonian (energy)**:
   
$$
H = -J \sum_{p} \prod_{\ell \in p} \sigma_\ell
$$

where each plaquette term is the product of 4 link variables around a square.


In [2]:
import tensorflow as tf

# For reproducibility
tf.random.set_seed(0)


## Step 1: Define the lattice and link variables

- We consider a lattice of size $L \times L$.
- Each site has two outgoing links:
  - $ U_x(i,j) $ : link in the x-direction
  - $ U_y(i,j) $ : link in the y-direction
- Therefore the link tensor has shape `(L, L, 2)`.
- Each link takes values in {+1, -1}.


In [3]:
L = 4
links = tf.where(
    tf.random.uniform((L, L, 2)) < 0.5,
    -tf.ones((L, L, 2), dtype=tf.int32),
    tf.ones((L, L, 2), dtype=tf.int32)
)

print("Links shape:", links.shape)
print("Links tensor (random ±1):\n", links.numpy())


I0000 00:00:1758151270.395170  104175 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1500 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Links shape: (4, 4, 2)
Links tensor (random ±1):
 [[[-1 -1]
  [ 1  1]
  [-1  1]
  [-1  1]]

 [[ 1 -1]
  [ 1  1]
  [ 1 -1]
  [-1  1]]

 [[ 1  1]
  [ 1 -1]
  [-1 -1]
  [ 1  1]]

 [[ 1  1]
  [ 1 -1]
  [-1  1]
  [-1 -1]]]


## Step 2: Extract components $ U_x $ and $ U_y $

We slice the last dimension of the tensor:

$$
U_x(i,j) = \texttt{links}[i,j,0],
\quad
U_y(i,j) = \texttt{links}[i,j,1]
$$


In [4]:
Ux = links[..., 0]  # shape (L, L)
Uy = links[..., 1]  # shape (L, L)

print("U_x:\n", Ux.numpy())
print("U_y:\n", Uy.numpy())

U_x:
 [[-1  1 -1 -1]
 [ 1  1  1 -1]
 [ 1  1 -1  1]
 [ 1  1 -1 -1]]
U_y:
 [[-1  1  1  1]
 [-1  1 -1  1]
 [ 1 -1 -1  1]
 [ 1 -1  1 -1]]


## Step 3: Construct the plaquette values

For a plaquette with bottom-left corner at $(i,j)$:

$$
P(i,j) = U_x(i,j) \; U_y(i+1,j) \; U_x(i,j+1) \; U_y(i,j)
$$

- We use `tf.roll` to shift tensors to implement periodic boundary conditions:
  - `Uy_shift_x = tf.roll(Uy, shift=-1, axis=0)` gives $ U_y(i+1,j) $.
  - `Ux_shift_y = tf.roll(Ux, shift=-1, axis=1)` gives $ U_x(i,j+1) $.


In [5]:
Uy_shift_x = tf.roll(Uy, shift=-1, axis=0)  # shift along x
Ux_shift_y = tf.roll(Ux, shift=-1, axis=1)  # shift along y

P = Ux * Uy_shift_x * Ux_shift_y * Uy
print("Plaquette values:\n", P.numpy())

Plaquette values:
 [[-1 -1 -1  1]
 [-1 -1 -1 -1]
 [ 1 -1  1 -1]
 [-1  1  1  1]]


## Step 4: Compute the Hamiltonian (energy)

The Hamiltonian is:

$$
H = -J \sum_{i,j} P(i,j)
$$

with coupling constant $J > 0$.  
Each plaquette contributes ±1 to the sum.


In [6]:
J = 1.0
energy = -J * tf.reduce_sum(tf.cast(P, tf.float32))

print("Total energy H =", energy.numpy())


Total energy H = 4.0
